In [2]:
import torch as t
import numpy as np
import preprocssing
from torch.utils.data import DataLoader, Dataset

c:\Users\omara\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
class TweetsDataset(Dataset):
    def __init__(self, csvPath):
        '''initialize the data and labels'''
        super().__init__()
        data, labels = preprocssing.preprocessDF(csvPath)
        self.data = data
        self.labels = labels
        self.size = len(data)
        self.data_padding()
    
    def __getitem__(self, index):
        ''' get the row item'''
        return self.data[index], self.labels[index]

    def data_padding(self):
        max_length = self.get_max_data_length()
        for data_row in self.data:
            if len(data_row) < max_length:
                padding = max_length - len(data_row)
                padding_list = [' '] * padding
                data_row += padding_list

    def get_max_data_length(self):
        max_length = -1e9
        for data_row in self.data:
            if len(data_row) > max_length:
                max_length = len(data_row)
        return max_length

    def __len__(self):
        '''get the dataset size'''
        return self.size

In [36]:
class TweetsClassifier(t.nn.Module):
    def __init__(self):
        pass
    def train(self):
        pass
    def predict(self):
        pass

In [ ]:
class RnnTweetsClassifier(TweetsClassifier):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, epoch_size, learning_rate, data_loader, embeddings = -1):
        super(RnnTweetsClassifier, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.epoch_size = epoch_size
        self.data_loader = data_loader
        self.embeddings = embeddings
        self.learning_rate = learning_rate
        self.rnn = t.nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = t.nn.Linear(hidden_size, num_classes)
        self.criterion = t.nn.CrossEntropyLoss()
        self.optimizer = t.optim.Adam(self.rnn.parameters(), self.learning_rate)
    
    def forward(self, input):
        #  h0 = t.zeros(self.num_layers, input.size(0), self.hidden_size)
         out, _ = self.rnn(input)
         out = out[:, -1, :]
         out = self.fc(out)
         return out

    def train(self):
        for epoch in range(self.epoch_size):
            for data, labels in self.data_loader:
                # reshape the row data
                tweet_size = data.shape[0]
                train_data = t.reshape(-1, tweet_size, self.input_size)

                output = self.rnn(train_data)
                


        
    
    def predict(self, tweet):
        pass

In [30]:
# load the dataset and dataloader
dataset = TweetsDataset('./dataset/train.csv')
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True)

[[('اتصالات', 'شاهد', 'تطعيم', 'اللقاح'), ('الجزائر', 'الأمير', 'كورونا', 'الأمريكي'), ('روسيا', 'تركي', ' ', 'امن'), ('لتصنيع', 'بن', ' ', 'ومفيد'), ('لقاح', 'هذلول', ' ', 'اصبحت'), ('سبوتنيك', 'يتلقى', ' ', 'الولايات'), ('v', 'الجرعه', ' ', 'المتحده'), ('الجزائر', 'الأولى', ' ', 'الأمريكيه'), (' ', 'لقاح', ' ', 'الأعلى'), (' ', 'كورونا', ' ', 'عدد'), (' ', ' ', ' ', 'الإصابات'), (' ', ' ', ' ', 'نريد'), (' ', ' ', ' ', 'لقاح'), (' ', ' ', ' ', 'امن'), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' ', ' '), (' ', ' ', ' '